## Data Analysis- Data cleaning / Pre-processing

Clean the country column and parse the date_measured column in the **store_income_data_task.csv** file.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
#Load up store store_income_date.csv
store_income = pd.read_csv("store_income_data_example.csv")

In [3]:
#check the data
store_income.head(5)

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,14 July 2006,UK
1,2,Nordson Corporation,NaN,Tools,$41744177.01,3 December 2006,united states of america
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12 August 2003,UNITED STATES
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,26 October 2006,UK
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,24 December 1973,UK


In [4]:
store_income.shape

(1000, 7)

In [5]:
store_income.columns

Index(['id', 'store_name', 'store_email', 'department', 'income',
       'date_measured', 'country'],
      dtype='object')

In [6]:
store_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   store_name     1000 non-null   object
 2   store_email    413 non-null    object
 3   department     973 non-null    object
 4   income         1000 non-null   object
 5   date_measured  1000 non-null   object
 6   country        1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [7]:
#look at the unique values in the "country" col
countries = store_income['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 34 unique countries


array(['UK ', 'united states of america', 'UNITED STATES', 'uk',
       ' United States of America', 'South Africa ', 'United States.',
       'United States', 'South Africa/', 'United States ',
       'United States of America', 'South Africa.', 'United Kingdom ',
       'United States of America ', 'United States of America/',
       'south africa', 'UK/', 'United Kingdom.', ' United Kingdom',
       ' South Africa', 'United Kingdom/', 'SOUTH AFRICA', ' UK',
       'united kingdom', 'UNITED KINGDOM', ' United States',
       'UNITED STATES OF AMERICA', 'South Africa', 'United States/',
       'united states', 'United States of America.', 'UK',
       'United Kingdom', 'UK.'], dtype=object)

In [8]:
#convecrt to lowercase and remove trailing white spaces
store_income['country'] = store_income['country'].str.lower()
store_income['country'] = store_income['country'].str.strip()

# Let us view the data
country = store_income['country'].unique()
print(f"There are {len(country)} unique countries")
country

There are 15 unique countries


array(['uk', 'united states of america', 'united states', 'south africa',
       'united states.', 'south africa/', 'south africa.',
       'united kingdom', 'united states of america/', 'uk/',
       'united kingdom.', 'united kingdom/', 'united states/',
       'united states of america.', 'uk.'], dtype=object)

In [9]:
#clean up the country column so there are three distinct countries
store_income['country'] = store_income['country'].str.replace(".","")

/tmp/ipykernel_3496/1026784093.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  store_income['country'] = store_income['country'].str.replace(".","")


In [10]:
#view if removed "."
store_income['country'].unique()

array(['uk', 'united states of america', 'united states', 'south africa',
       'south africa/', 'united kingdom', 'united states of america/',
       'uk/', 'united kingdom/', 'united states/'], dtype=object)

In [11]:
store_income['country'] = store_income['country'].str.replace("/", "")
store_income['country'].unique()

array(['uk', 'united states of america', 'united states', 'south africa',
       'united kingdom'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [12]:
# helpful libraries
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# set seed for reproducibility
np.random.seed(0)

# get the top 10 closest matches to "united kingdom"
matches_uk = fuzzywuzzy.process.extract("uk", country, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches_uk

[('uk', 100),
 ('uk/', 100),
 ('uk.', 100),
 ('south africa', 14),
 ('south africa/', 14),
 ('south africa.', 14),
 ('united states', 13),
 ('united states.', 13),
 ('united states/', 13),
 ('united kingdom', 12)]

In [13]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_col(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match,
                                         strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    # only get matches with a ratio >90
    close_matches = [matches[0] for matches in matches if matches[1] >=min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

    # let us know the function is done
    print("All done")

In [14]:
replace_matches_in_col(df=store_income, column='country', string_to_match="united kingdom")
replace_matches_in_col(df=store_income, column='country', string_to_match="united states")
replace_matches_in_col(df=store_income, column='country', string_to_match="united states of america")
replace_matches_in_col(df=store_income, column='country', string_to_match="south africa")
replace_matches_in_col(df=store_income, column='country', string_to_match="uk")

All done
All done
All done
All done
All done


In [15]:
# get all the unique values in the 'country' column
countries = store_income['country'].unique()

print(f"There are {len(countries)} unique countries")
countries


There are 5 unique countries


array(['uk', 'united states of america', 'united states', 'south africa',
       'united kingdom'], dtype=object)

In [16]:
store_income.replace('uk', 'United_Kingdom', inplace=True)
store_income.replace('england', 'United_Kingdom', inplace=True)
store_income.replace('britain', 'United_Kingdom', inplace=True)
store_income.replace('united kingdom', 'United_Kingdom', inplace=True)
store_income.replace('sa', 'South_Africa', inplace=True)
store_income.replace('s africasouth africa', 'South_Africa', inplace=True)
store_income.replace('united states of america', 'United_States', inplace=True)
store_income.replace('united states', 'United_States', inplace=True)
store_income.replace('america', 'United_States', inplace=True)

# get all the unique values in the 'country' column
countries = store_income['country'].unique()

print(f"There are {len(countries)} unique countries")
countries

There are 3 unique countries


array(['United_Kingdom', 'United_States', 'south africa'], dtype=object)

In [17]:
#Fill empty values with NaN and then
store_income.replace('', np.nan, inplace=True)
store_income.replace('nan', np.nan, inplace=True)

store_income["country"].unique()
#There are only 3 contries as shown and the empty values
#if needed we can drop the empty NaN values using the following:
#store_income.dropna(subset = ["country"],inplace=True) without inplace=True will drop the whole row of the missing values



array(['United_Kingdom', 'United_States', 'south africa'], dtype=object)

In [18]:
store_income.head(5)

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,14 July 2006,United_Kingdom
1,2,Nordson Corporation,NaN,Tools,$41744177.01,3 December 2006,United_States
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12 August 2003,United_States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,26 October 2006,United_Kingdom
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,24 December 1973,United_Kingdom


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column, but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [28]:
# Convert 'date_measured' column to datetime objects 
# Copy the "date_measured column" and name it "days_ago"

# module we 'll use
from datetime import date
store_income['days_ago'] = pd.to_datetime(store_income['date_measured'], infer_datetime_format=True)

store_income['date_measured'] = pd.to_datetime(store_income['date_measured'], format='%d %B %Y')


In [31]:
#store_income['days_ago'].head(5)
store_income["days_ago"].dtype
# '<M8[ns]'type is the same as 'datetime64[ns]'

dtype('<M8[ns]')

In [53]:
# Calculate the number of days ago for each date
import datetime
from datetime import datetime, date

#Create a function to convert datetime onject:
#def convert_to_datetime(date):
#    #for date in date_int:
#    date = str(date)
#    #if date != "%Y-%m-%d":
#    return datetime.strptime(date, '%d-%m-%Y')
 
# Convert the specific date string to a datetime object
#store_income['days_ago'] = convert_to_datetime(store_income['days_ago'])

#Convert everything to pandas type 
# and extract the times for the comparison
#Ensure that dtypes of the columns are something
# like datetime64[ns] and timedelta64[ns]. 
# For that, try converting them explicitly using 
# pd.to_datetime and pd.to_timedelta.
#Find the current day
current_day = pd.to_datetime(date.today())
#store it in the col 
store_income["days_ago"] = current_day
#Calculate the number of days passed from the current day
print(store_income["date_measured"]-store_income["days_ago"])


0      -6547 days
1      -6405 days
2      -7614 days
3      -6443 days
4     -18437 days
          ...    
995    -6831 days
996   -12347 days
997    -5488 days
998    -4755 days
999    -4700 days
Length: 1000, dtype: timedelta64[ns]
